In [2]:
import time
from modules import device, model, context_input, response_input, embs_gen, cand_emb_gen, score, get_cand_embs, response_table

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
cand_embs = get_cand_embs()
response_table;

In [51]:
response_table['response'][0]

"['좋은 아빠가 되실 거 같아요. 진심으로 축하드려요.']"

In [29]:
def chatbot(query):
    query = [query]
    embs = embs_gen(*context_input(query))
    score_lst = score(embs, cand_embs)
    idx = score_lst.argmax(1)
    idx = int(idx[0])
    return response_table['response'][idx]

In [30]:
chatbot('오늘 완전 기분 최악이야')

"['너무 화나시겠어요.']"

In [31]:
response_table['response'][1]

"['약을 잘 챙겨 먹어 건강해지시길 바랄게요.']"

In [64]:
import pickle

with open('./data/train_data_source.pickle', 'rb') as f:
    train = pickle.load(f)
with open('./data/val_data_source.pickle', 'rb') as f:
    dev = pickle.load(f)

In [65]:
train[0]['responses'][0]

'좋은 아빠가 되실 거 같아요. 진심으로 축하드려요.'

In [66]:
data = {
    'context' : [],
    'response': []
}

In [67]:
for sample in train:
    data['context'].append(sample['context'])
    data['response'].append([sample['responses'][0]])

In [68]:
import pandas as pd
df = pd.DataFrame(data)

In [69]:
df.to_pickle('response_data.pickle')

In [71]:
df = pd.read_pickle('response_data.pickle')

In [73]:
type(df['response'][0])

list